In [1]:
# import libraries
import music21 as m21
import pandas as pd
from composer_toolkit import set_theory as st

In [2]:
sets = {
    'bar 1': {
        'M7_1': 'c f# c#',
        '1': 'c g d f# c#',
        '2': 'e f# g b d d#',
        '1u2': 'c g d f# c# e b d#',
        '3': 'g# d g b d# a#',
        '3uM': 'g# d g b d# a# f e'
    },
    'bar 2': {
        '1': 'bb a db g ab c',
        '2': 'eb f c db gb bb',
        '1u2': 'bb a db g ab c eb f gb',
        '3': 'a eb ab d g db',
        '3uM': 'a eb ab d g db b e', 
    },
    'bar 3': {
        '1': 'db a c b',
        '2': 'd f gb bb eb',
        '1u2': 'db a c b d f gb bb eb',
        '3': 'bb e a eb cb ab c',
        'M': 'f c cb db d',
        '3u4': 'bb e a eb cb ab c f db d',
    },
    'bar 4': {
        '0': 'g f# c f bb',
        '1': 'a g# d g',
        '2': 'd f# b# c# g bb',
        '1u2': 'a g# d g f# b# c# bb',
    },
    'bar 4-5': {
        '0': 'd g# b e# f# d# f e',
        '1': 'e bb b f# a eb',
        '2': 'a d g# d# g',
        '1u2': 'e bb b f# a d g# d# g',
    },
    'bar 6': {
        'cl-b': 'g# a c',
        'cl-t': 'eb g ab d',
        'cl': 'g# a c eb g ab d',
    },
    'bar 7': {
        'cl': 'c eb g ab d',
    },
    'bar 8': {
        '1(-M)': 'b f# c# f',
        '1': 'b f# c# f c',
        '2': 'a c db f gb bb',
        '1u2': 'b f# c# f c a db gb bb',
        '3': 'eb g ab d',
        '3uM': 'eb g ab d cb bb'
    },
    'bar 9': {
        '1': 'ab d g f#',
        '2': 'eb g f# bb d',
        '1u2': 'ab d g f# eb bb',
    },
    'bar 10': {
        'M7_1': 'f# c f',
        '1': 'f# c f e',
        '2': 'f# a# d f g c#',
        '1u2': 'f# c f e a# d g c#',
    },
    'bar 11': {
        'M7_1': 'f# c f',
        'M7_2': 'g d g#',
        'M7_3': 'a d# g#',
        '0': 'f# c f a d# g#',
        '1': 'g d g# c# f#',
        '2': 'c# g# d c f# g',
        '1u2': 'g d g# c# f# c',
        '3': 'a d# g# a# c# g f#'
    },
    'bar 12': {
        '0': 'b f# g f c# c',
        '1u2': 'eb d g c f ab cb'
    },
    'bar 13 (1-3)': {
        '3': 'b f# g# bb f a',
        '3uM': 'b f# g# bb f a e eb',
    },
    'bar 13 (3-5)': {
        '1': 'a ab c f# g b',
        '2': 'c e b f a',
        '1u2': 'a ab c f# g b e f',
        '3': 'ab d g f# c',
        '3uM': 'ab d g f# c bb eb',
    },
    'bar 14': {
        '1': 'c ab b bb',
        '2': 'db e f a d',
        '1u2': 'c ab b bb db e f a d',
        '3': 'a eb ab bb d g b',
        'M': 'e b bb c db',
        '3uM': 'a eb ab bb d g b e c db',
    },
    'bar 15': {
        '0': 'gb f b e a',
        '1': 'ab g c# f#',
        '2': 'c# e# b c f# a',
        '1u2': 'ab g c# f# e# b c a'
    },
    'bar 15-16': {
        '0': 'f e eb a# d',
        '0.1': 'c a db ab',
        '0.2': 'c g b d f# c#',
        '1': 'bb e b eb',
        '2': 'a eb ab d c',
        '1u2': 'bb e b eb a ab d c',
    },
    'bar 17': {
        'cl': 'g c# f c',
    },
    'bar 18': {
        'cl-b': 'd a eb g c#',
        'cl-t': 'd c# g',
        'cl': 'd a eb g c#',
    },
    'bar 19': {
        'cl-b': 'd a eb g c#',
        'cl-t': 'f# c f a d# g#',
        'cl': 'd a eb g c# f# c f g#'
    }
}

sets_forte = {}

for group in sets:
    sets_forte[group] = {}
    for s in sets[group]:
        sets_forte[group][s] = st.forte_class(sets[group][s].split())


In [9]:
data_forte = pd.DataFrame.from_dict(sets_forte)
counts = data_forte.replace(to_replace='A|B', value='', regex=True)
counts = counts.apply(pd.Series.value_counts)
#data_forte = data_forte.transpose()
total_counts = counts.sum(1)
total_counts = total_counts.to_frame()
total_counts.rename(columns={0: 'occurance'}, inplace=True)

In [10]:
sets_subsets = {}

# calculate subsets
for group in sets:
    sets_subsets[group] = {}
    for s in sets[group]:
        chord = m21.chord.Chord(sets[group][s].split())
        subsets = st.subsets(chord)
        sets_subsets[group][s] = subsets

# count most common subsets
counts_subsets = {}
for group in sets_subsets:
    for s in sets_subsets[group]:
        for subset in sets_subsets[group][s]:
            if subset in counts_subsets:
                counts_subsets[subset] += 1
            else:
                counts_subsets[subset] = 1
                
# sort subsets by occurance
counts_subsets = sorted(counts_subsets.items(), key=lambda x: x[1], reverse=True)

# to dataframe
counts_subsets = pd.DataFrame(counts_subsets, columns=['subset', 'occurance'])

    

In [5]:
# search for subset and identify which sets contain it
def search_subset(subset):
    sets_contain = {}
    for group in sets_subsets:
        for s in sets_subsets[group]:
            if subset in sets_subsets[group][s]:
                if group in sets_contain:
                    sets_contain[group].append(s)
                else:
                    sets_contain[group] = [s]
    return sets_contain



In [6]:
counts_subsets

,subset,occurance
0,3-5,68
1,3-4,68
2,3-8,63
3,3-9,57
4,3-1,56
...,...,...
196,8-27,1
197,8-26,1
198,8-23,1
199,7-34,1


In [7]:
search_subset('3-4')

{'bar 1': ['1', '2', '1u2', '3', '3uM'],
 'bar 2': ['1', '2', '1u2', '3', '3uM'],
 'bar 3': ['2', '1u2', '3', 'M', '3u4'],
 'bar 4': ['0', '2', '1u2'],
 'bar 4-5': ['0', '1', '2', '1u2'],
 'bar 6': ['cl-t', 'cl'],
 'bar 7': ['cl'],
 'bar 8': ['1(-M)', '1', '2', '1u2', '3', '3uM'],
 'bar 9': ['1', '2', '1u2'],
 'bar 10': ['1', '2', '1u2'],
 'bar 11': ['1', '2', '1u2', '3'],
 'bar 12': ['0', '1u2'],
 'bar 13 (1-3)': ['3', '3uM'],
 'bar 13 (3-5)': ['1', '2', '1u2', '3', '3uM'],
 'bar 14': ['2', '1u2', '3', 'M', '3uM'],
 'bar 15': ['0', '2', '1u2'],
 'bar 15-16': ['0', '0.1', '0.2', '1', '1u2'],
 'bar 17': ['cl'],
 'bar 18': ['cl-b', 'cl'],
 'bar 19': ['cl-b', 'cl']}

In [8]:
search_subset('3-5')

{'bar 1': ['1', '1u2', '3', '3uM'],
 'bar 2': ['1', '2', '1u2', '3', '3uM'],
 'bar 3': ['1u2', '3', 'M', '3u4'],
 'bar 4': ['0', '1', '2', '1u2'],
 'bar 4-5': ['0', '1', '2', '1u2'],
 'bar 6': ['cl-t', 'cl'],
 'bar 7': ['cl'],
 'bar 8': ['1(-M)', '1', '2', '1u2', '3', '3uM'],
 'bar 9': ['1', '1u2'],
 'bar 10': ['1', '2', '1u2'],
 'bar 11': ['0', '1', '2', '1u2', '3'],
 'bar 12': ['0', '1u2'],
 'bar 13 (1-3)': ['3', '3uM'],
 'bar 13 (3-5)': ['1', '2', '1u2', '3', '3uM'],
 'bar 14': ['1u2', '3', 'M', '3uM'],
 'bar 15': ['0', '1', '2', '1u2'],
 'bar 15-16': ['0', '0.2', '1', '2', '1u2'],
 'bar 17': ['cl'],
 'bar 18': ['cl-b', 'cl'],
 'bar 19': ['cl-b', 'cl-t', 'cl']}